# PVT свойства модели Black Oil по uniflocvba и uniflocpy

In [ ]:
import sys
sys.path.append('../')


import uniflocpy.uTools.data_workflow as data_workflow
import pandas as pd
from uniflocpy.uTools import plotly_workflow
import uniflocpy.uPVT.BlackOil_model as BlackOil_model
import uniflocpy.uValidation.python_api as python_api
import uniflocpy.uTools.uconst as uc

In [ ]:
def create_result_df_from_vba_output(vba_result, p_bar, t_c):
    result_dict = {}
    for key, value in zip(vba_result[1],vba_result[0]):
        result_dict[key] = value
    result_dict['p_bar'] = p_bar
    result_dict['t_c'] = t_c
    this_df = pd.DataFrame(result_dict, index = [0])
    this_df['pb_bar'] = uc.atm2bar(this_df['pb_atma'])
    return this_df

Загрузка надстройки

In [ ]:
uniflocvba = python_api.API('E:\\Git\\unifloc_vba\\UniflocVBA_7.xlam')

Выбор параметров

In [ ]:
gamma_oil = 0.86
gamma_water = 1
gamma_gas = 1.2
rsb_m3m3 = 56
t_res_c = 40
t_c = 30
p_bar = 90
#rp_m3m3 = 80
#pb_cal_bar = 120
#bob_cal_m3m3 = 1.2
#mu_oil_bubble_cp = 1

keywords_python = {"gamma_oil": gamma_oil, "gamma_gas": gamma_gas, "gamma_wat":gamma_water,
                                    "rsb_m3m3": rsb_m3m3, "t_res_c": t_res_c}

keywords_vba = {"t_C": t_c, "gamma_gas": gamma_gas,
                "gamma_oil": gamma_oil, "gamma_wat": gamma_water, "rsb_m3m3": rsb_m3m3, "tres_C": t_res_c,
               "PVTcorr":1} #проверено, улетают свойства газа (из-за z), st oil-gas, mu_wate



Настройка модели флюида

In [ ]:

blackoil_option = BlackOil_model.BlackOil_option()
#blackoil_option.b_wat_cor_number = 1
#blackoil_option.mu_wat_cor_number = 1
#blackoil_option.rho_wat_cor_number = 1
#blackoil_option.z_cor_number = 1
#blackoil_option.pseudocritical_temperature_cor_number = 1
#blackoil_option.pseudocritical_pressure_cor_number = 1
#blackoil_option.rho_gas_cor_number = 1
#blackoil_option.b_gas_cor_number = 1
#blackoil_option.mu_dead_oil_cor_number = 2
#blackoil_option.sigma_oil_gas_cor_number = 2
#blackoil_option.sigma_wat_gas_cor_number = 1

blackoil_option.set_vba_preset()

#unf_pvt_compressibility_oil_VB_1atm
blackoil_option.pb_cor_number = 1
blackoil_option.mu_dead_oil_cor_number = 1
blackoil_option.rs_cor_number = 1
blackoil_option.rho_oil_cor_number = 1

blackoil_option.b_oil_below_pb_cor_number = 2
blackoil_option.b_oil_above_pb_cor_number = 1
blackoil_option.mu_oil_pb_cor_number = 1
blackoil_option.mu_oil_any_p_cor_number = 1
#blackoil_option.compr_oil_cor_number = 1
#blackoil_option.compr_oil_below_pb_cor_number = 1
python_fluid = BlackOil_model.Fluid(**keywords_python, option = blackoil_option)


Расчет двух моделей

In [ ]:
BlackOil_model_db = data_workflow.Data()
BlackOil_model_db.clear_data()
vba_result_df = None
for p_bar in range(2, 250, 1):
    python_fluid.calc(p_bar, t_c)
    p_atm = uc.bar2atm(p_bar)
    BlackOil_model_db.get_data(python_fluid, object_name='python_fluid')
    vba_result = uniflocvba.PVT_all_pvt(p_atm, **keywords_vba)
    this_vba_result_df = create_result_df_from_vba_output(vba_result, p_bar, t_c)
    try:
        vba_result_df = vba_result_df.append(this_vba_result_df)
    except:
        vba_result_df = this_vba_result_df.copy()

Объединение результатов

In [ ]:
result_unifloc_python = BlackOil_model_db.get_data_as_df()
result_unifloc_python.index = result_unifloc_python['python_fluid.p_bar']
result_unifloc_python.index.name = 'p_bar'

vba_result_df.index = result_unifloc_python.index
vba_result_df.index.name = 'p_bar'
vba_result_df = vba_result_df.add_prefix('vba_fluid.')
all_result = result_unifloc_python.join(vba_result_df, rsuffix=' (uniflocvba)')

Построение графиков

In [ ]:
plotly_workflow.plot_by_patterns(all_result, ['pb'], ['cor_number', 'cal', 'mpa', 'tpb'], swap_xy=False, reversed_y=False,  plot_name='Распределение PVT свойств от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result, ['rs'], ['cor_number', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение PVT свойств от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result, ['mu'], ['cor_number', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение PVT свойств от давления',
                                x_name = 'Давление, бар', y_name = 'Вязкость, сПуаз')

In [ ]:
plotly_workflow.plot_by_patterns(all_result, ['rho_oil'], ['cor_number', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение PVT свойств от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result, ['rho_oil'], ['cor_number', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение PVT свойств от давления',
                        traces_names = ['Плотность дегазированной нефти, кгм/м3', 'Плотность питона', 'Плотность вба'])

In [ ]:
plotly_workflow.plot_by_patterns(all_result, ['z'], ['cor_number', 'cal', 'mpa', 'tpb'], swap_xy=False, reversed_y=False,  plot_name='Распределение PVT свойств от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result, [['bo','m3m3'],['b_oil','m3m3']], ['cor_number', 'ob', '_b_','cal', 'mpa', 'tpb', 'rs'], swap_xy=False, reversed_y=False,  plot_name='Распределение PVT свойств от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result, ['sigma'], ['cor_number', 'cal', 'mpa', 'tpb', 'rs'], swap_xy=False, reversed_y=False,  plot_name='Распределение PVT свойств от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result, ['comp'], ['cor_number', 'cal', 'mpa', 'tpb', 'rs'], swap_xy=False, reversed_y=False,  plot_name='Распределение PVT свойств от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [["heatcap"], ["JkgC"]], ['fl.','cor_number', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение PVT свойств от давления')